#Hindi ULMFiT from scratch based on Rachel/Jeremy's awesome [Code-First Intro to Natural Language Processing](https://www.youtube.com/playlist?list=PL8l5P33wvCvIqO3UyvdkeRMoGg--VBvho) and its [Notebook](https://github.com/fastai/course-nlp/blob/master/nn-vietnamese.ipynb)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
# bs=48
# bs=24
bs=128

In [3]:
torch.cuda.set_device(0)

In [4]:
data_path = Config.data_path()
print(data_path)

/home/ash022/.fastai/data


/opt/conda/lib/python3.6/site-packages/fastai/datasets.py:164: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  with open(fpath, 'r') as yaml_file: return yaml.load(yaml_file)


This will create a `viwiki` folder, containing a `viwiki` text file with the wikipedia contents. (For other languages, replace `vi` with the appropriate code from the [list of wikipedias](https://meta.wikimedia.org/wiki/List_of_Wikipedias).)

In [5]:
lang = 'hi'
#lang = 'vi'
# lang = 'zh'

In [6]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

## Hindi wikipedia model

### Download data

In [7]:
from nlputils import split_wiki,get_wiki

In [8]:
get_wiki(path,lang)

unzipping...


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [9]:
!bunzip2 /home/ash022/.fastai/data/hiwiki/hiwiki-latest-pages-articles.xml.bz2
!git clone https://github.com/fonnesbeck/Bios8366.git
!python WikiExtractor.py --processes 4 --no_templates  --min_text_length 1800 --filter_disambig_pages --log_file log -b 100G -q $HOME/.fastai/data/hiwiki/hiwiki-latest-pages-articles.xml
!mv text/AA/wiki_00 $HOME/.fastai/data/hiwiki/hiwiki

In [17]:
name=path.ls()

In [8]:
!head -n4 /home/ash022/.fastai/data/hiwiki/hiwiki-latest-pages-articles.xml

<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="hi">
  <siteinfo>
    <sitename>विकिपीडिया</sitename>
    <dbname>hiwiki</dbname>


This function splits the single wikipedia file into a separate file per article. This is often easier to work with.

In [12]:
dest = split_wiki(path,lang)

/home/ash022/.fastai/data/hiwiki/docs already exists; not splitting


In [13]:
dest.ls()[:5]

[PosixPath('/home/ash022/.fastai/data/hiwiki/docs/पाड़.txt'),
 PosixPath('/home/ash022/.fastai/data/hiwiki/docs/इलाहाबाद.txt'),
 PosixPath('/home/ash022/.fastai/data/hiwiki/docs/सैयद अता हसनैन.txt'),
 PosixPath('/home/ash022/.fastai/data/hiwiki/docs/रविन्द्र चौबे.txt'),
 PosixPath('/home/ash022/.fastai/data/hiwiki/docs/मैरी कॉम.txt')]

In [ ]:
# Use this to convert Chinese traditional to simplified characters
# ls *.txt | parallel -I% opencc -i % -o ../zhsdocs/% -c t2s.json

### Create pretrained model

In [14]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(60004, 15147)

In [16]:
print(path,f'{lang}_databunch')
!mv /home/ash022/.fastai/data/hiwiki/docs/hi_databunch /home/ash022/.fastai/data/hiwiki/hi_databunch

/home/ash022/.fastai/data/hiwiki hi_databunch


In [35]:
#data = load_data(path, f'{lang}_databunch', bs=bs)
#data = load_data(path, 'docs', bs=bs)
torch.cuda.empty_cache()
language_model_learner()
learn.destroy()

TypeError: language_model_learner() missing 2 required positional arguments: 'data' and 'arch'

In [1]:
import tensorflow as tf
from numba import cuda

a = tf.constant([1.0,2.0,3.0],shape=[3],name='a')
b = tf.constant([1.0,2.0,3.0],shape=[3],name='b')
with tf.device('/gpu:1'):
    c = a+b

TF_CONFIG = tf.ConfigProto(
gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.1),
  allow_soft_placement=True)

sess = tf.Session(config=TF_CONFIG)
sess.run(tf.global_variables_initializer())
i=1
while(i<1000):
        i=i+1
        print(sess.run(c))

sess.close()
cuda.select_device(1)
cuda.close()
with tf.device('/gpu:1'):
    c = a+b

TF_CONFIG = tf.ConfigProto(
gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5),
  allow_soft_placement=True)

sess = tf.Session(config=TF_CONFIG)

sess.run(tf.global_variables_initializer())
while(1):
        print(sess.run(c))

[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]
[2. 4. 6.]

IndexError: list index out of range

In [73]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False)#.to_fp16()

In [74]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time


Save the pretrained model and vocab:

In [18]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

## sentiment analysis

### Language model

- [Data](https://github.com/shubham721/Sentiment-Analysis-On-Hindi-Reviews)

In [42]:
pos_df = pd.read_table('pos_hindi.txt',header=None)
pos_df.columns=['comment']
pos_df['label']=1
#train_df.loc[pd.isna(train_df.comment),'comment']='NA'
pos_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,comment,label
0,मिकी वाइरस पूरी तरह से मनीष पॉल की फिल्म है और...,1
1,$मनीष ने अपने किरकार को बखूबी निभाया है,1
2,"$उन्होंने शाहिद के डर, खौफ, संघर्ष, जिद और जीत...",1
3,$वे हर भाव के दृश्यों में नैचुरल लगते हैं,1
4,$फिल्म की कास्टिंग जबरदस्त है,1


In [43]:
neg_df = pd.read_table('neg_hindi.txt',header=None,sep="\n")
neg_df.columns=['comment']
neg_df['label']=0
#test_df.loc[pd.isna(test_df.comment),'comment']='NA'
neg_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,comment,label
0,फिल्म देखने के बाद न सिर्फ उम्मीदें धराशायी हु...,0
1,$उनके अभिनय में इंटेसिंटी तो है लेकिन किरदार स...,0
2,$एली अबराम को फिल्म में महज ग्लैमर ऐड करने के ...,0
3,$ बाकी के भी किरदार अति सामान्य हैं,0
4,"$ संदर्भ और स्थितियां बदल गई हैं, लेकिन सोच और...",0


In [55]:
df = pd.concat([pos_df,neg_df], sort=False)
df.head(), df.tail()

(                                             comment  label
 0  मिकी वाइरस पूरी तरह से मनीष पॉल की फिल्म है और...      1
 1            $मनीष ने अपने किरकार को बखूबी निभाया है      1
 2  $उन्होंने शाहिद के डर, खौफ, संघर्ष, जिद और जीत...      1
 3          $वे हर भाव के दृश्यों में नैचुरल लगते हैं      1
 4                      $फिल्म की कास्टिंग जबरदस्त है      1,
                                                comment  label
 506  $अगर डायरेक्टर बार - बार हीरो राम द्वारा घर की...      0
 507     $मैं गलियों से निकलता हूं फिर भी फंस जाता हूं।      0
 508     $क्लाइमेक्स भी बेवजह खींचा गया महसूस होता है।       0
 509  $हम इस धारणा के प्रभाव को महसूस करते हैं, लेकि...      0
 510                                                  $      0)

In [56]:
data_lm = (TextList.from_df(df, path, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [57]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

/opt/conda/lib/python3.6/site-packages/fastai/datasets.py:164: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  with open(fpath, 'r') as yaml_file: return yaml.load(yaml_file)


In [58]:
lr = 1e-3
lr *= bs/48

In [59]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.137067,4.378086,0.276228,00:00
1,4.962988,4.295090,0.279464,00:00


In [60]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.638175,4.205736,0.272768,00:00
1,4.601702,4.058837,0.273326,00:00
2,4.526342,3.834914,0.288170,00:00
3,4.425710,3.666122,0.305580,00:00
4,4.316147,3.554887,0.315290,00:00
5,4.215147,3.546034,0.326004,00:00
6,4.131269,3.513413,0.331920,00:00
7,4.072524,3.486128,0.333147,00:00


In [61]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [62]:
data_clas = (TextList.from_df(pos_df, path, vocab=data_lm.vocab, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [63]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [67]:
from sklearn.metrics import f1_score
import numpy as np
#@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [68]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

/opt/conda/lib/python3.6/site-packages/fastai/datasets.py:164: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  with open(fpath, 'r') as yaml_file: return yaml.load(yaml_file)


In [69]:
lr=2e-2
lr *= bs/48

In [72]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7)).to_fp16()

epoch,train_loss,valid_loss,accuracy,f1,time


RuntimeError: output with type torch.cuda.HalfTensor doesn't match the desired type torch.cuda.FloatTensor

In [46]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.321768,0.255457,0.899254,0.871367,00:02
1,0.305934,0.250888,0.894901,0.872021,00:02


In [47]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.300939,0.261080,0.893657,0.866201,00:03
1,0.263790,0.220207,0.906716,0.886115,00:03


In [48]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.282888,0.238203,0.905473,0.886483,00:04
1,0.248599,0.216489,0.918532,0.901550,00:04


In [49]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.201508,0.217176,0.911070,0.890084,00:05


In [50]:
learn_c.save(f'{lang}clas')

Competition top 3 f1 scores: 0.90, 0.89, 0.89. Winner used an ensemble of 4 models: TextCNN, VDCNN, HARNN, and SARNN.

## Ensemble

In [65]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load(f'{lang}clas', purge=False);

In [69]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)

(tensor(0.9111), tensor(0.8952))

In [67]:
data_clas_bwd = load_data(path, f'{lang}_textlist_class_bwd', bs=bs, num_workers=1, backwards=True)
learn_c_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c_bwd.load(f'{lang}clas_bwd', purge=False);

In [70]:
preds_b,targs_b = learn_c_bwd.get_preds(ordered=True)
accuracy(preds_b,targs_b),f1(preds_b,targs_b)

(tensor(0.9092), tensor(0.8957))

In [71]:
preds_avg = (preds+preds_b)/2

In [72]:
accuracy(preds_avg,targs_b),f1(preds_avg,targs_b)

(tensor(0.9154), tensor(0.9016))

## Future work
* Try [SentencePiece](https://github.com/google/sentencepiece)
* Check activation with [Interpretability Beyond Feature Attribution: Quantitative Testing with Concept Activation Vectors](https://github.com/tensorflow/tcav)
* [GPT2](https://gist.github.com/mohdsanadzakirizvi/f1419fba3907af2baf6e0f6ab2a53d5b)
* [Unsupervised word embeddings capture latent knowledge from materials science literature](https://www.nature.com/articles/s41586-019-1335-8)
